In [1]:
#import libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pylab
from pylab import *
import sys
import time
import scipy
from scipy import optimize 
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image

# variable initialization to detect the number of rectangles in the mouth contour region for yawn  
r = 0
# variable initialization to store yawn time 
global yst
yst = 0
# variable initialization to verify the occurance of yawn
global flag
flag = True
# List initialization to hold the number of yawns in a particular period to reinitialise the list
yrc = []
# variable initialization to count the number of yawns
cnt = 0
# variable initialization to measure the time taken to yawn
avg_yt = 3

In [3]:
cascade_face = cv2.CascadeClassifier(r"C:\Users\prart\OneDrive\Desktop\AMLProject\HC\HaarCascade\haarcascade_frontalface_default.xml")
cascade_eyes = cv2.CascadeClassifier(r"C:\Users\prart\OneDrive\Desktop\AMLProject\HC\HaarCascade\haarcascade_eye_tree_eyeglasses.xml")
model_import = load_model(r"C:\Users\prart\OneDrive\Desktop\AMLProject\BestModel")

In [4]:
# Computing the contours of the face for eye and mouth detection
def contour_compute(img, contour):
    cv2.drawContours(img, contour, -1, (0,255,0), 3)
    area_maximum = 0
    max2 = 0
    cnt_max = 0
    cnt_max2 = 0
    for c in contour:
        cnt = c
        ta = cv2.contourArea(cnt)
        if area_maximum < ta:
            max2 = area_maximum
            area_maximum = ta
            cnt_max2 = cnt_max
            cnt_max = cnt
        elif (max2 < ta):
            max2 = ta
            cnt_max2 = cnt
    return [cnt_max2, max2]

# Function initialization to determine the region of contour for extraction of the mouth region to detect a yawn
def contour_threshold(mouth, area_rectangle):
    global r
# Histogram equalization function and grayscaling of the input frame
    grey_image = cv2.equalizeHist(cv2.cvtColor(mouth, cv2.COLOR_BGR2GRAY))
    x,img_threshholding = cv2.threshold(grey_image, 64, 255, cv2.THRESH_BINARY)
    contour, ordering = cv2.findContours(img_threshholding, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    returnValue = contour_compute(mouth, contour)
    cnt_max2 = returnValue[0]
    contourArea = returnValue[1]
    r = contourArea / area_rectangle
    if(isinstance(cnt_max2, np.ndarray) and len(cnt_max2) > 0):
        cv2.drawContours(mouth, [cnt_max2], 0, (255,0,0), -1)


In [ ]:
# Initialization of the variable to store the video that is captured by the webcam
webcam_on = cv2.VideoCapture(0)
while True:
    z, frame = webcam_on.read()
    horizontal = frame.shape[0]
    vertical = frame.shape[1]
#Histogram equalization on the frame along with grayscaling the input frame of the video
    he = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    histogram,bins = np.histogram(frame.flatten(),256,[0,256])
    cummulative_df = histogram.cumsum()
    cdf_normalized = cummulative_df * float(histogram.max()) / cummulative_df.max()
    image_grey = cv2.equalizeHist(he)
#Extraction of the face feature from the input frame
    img_face= cascade_face.detectMultiScale(image_grey, scaleFactor= 1.2, minNeighbors=3)
#Extraction of the eye feature from the input frame
    img_eye= cascade_eyes.detectMultiScale(image_grey, scaleFactor= 1.1, minNeighbors=1)
    cv2.rectangle(frame, (0,horizontal-50),(350,horizontal),(0,0,0),thickness=cv2.FILLED)
    for (start_x,start_y,width,height) in img_face:
        cv2.rectangle(frame,pt1=(start_x,start_y),pt2=(start_x+width,start_y+height), color= (255,0,0), thickness=3 )
#Extraction of the contour region for mouth from the input frame       
        start_width = int((start_x + (width / 4)))
        end_width = int(start_x + ((3 * width) / 4))
        start_height = int(start_y + (11 * height / 16))
        end_height = int(start_y + height)
        cv2.rectangle(frame, pt1=(start_width, start_height), pt2=(end_width, end_height),color=(255,0,0),thickness= 3)
#Extraction of the mouth feature from the input frame
        mouth = frame[start_height:end_height, start_width:end_width]
        area_rectangle = (width*height)/2
        if(len(mouth) > 0):
            contour_threshold(mouth, area_rectangle)
        if(r > 0.06):
            if(flag is True):
                flag = False
                yst = time.time()
#Check to see if the duration of the yawn is greater than or equal to 3 seconds
            if((time.time() - yst) >= avg_yt):
                cnt += 1
                yrc.append(cnt)
# Printing "Yawn detected" on the display window of the frame if the check is satisfied       
                cv2.putText(frame,'Yawn Detected',(160,horizontal-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(0,255,0),
                       thickness=1,lineType=cv2.LINE_AA)
# Condition check to reinitialise the yawn detection variables to determine if there is a yawn again
                if(len(yrc) > 6):
                    flag = True
                    yst = 0      
    for (start_eye_x,start_eye_y,width_eye,height_eye) in img_eye:
        cv2.rectangle(frame,pt1=(start_eye_x,start_eye_y),pt2=(start_eye_x+width_eye,start_eye_y+height_eye), color= (255,0,0), thickness=3 )
# Model prediction for open or closed eyes
        img_eye= frame[start_eye_y:start_eye_y+height_eye,start_eye_x:start_eye_x+width_eye]
        img_eye= cv2.resize(img_eye,(80,80))
        img_eye= img_eye/255
        img_eye= img_eye.reshape(80,80,3)
        img_eye= np.expand_dims(img_eye,axis=0)
        eye_pred = model_import.predict(img_eye)
# Closed eye prediction
        if eye_pred[0][0]>0.30:
            cv2.putText(frame,'Eyes closed',(10,horizontal-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)
# Open eye prediction
        elif eye_pred[0][1]>0.90:
            cv2.putText(frame,'Eyes open',(10,horizontal-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)   
# Display of the final output frame with state of the eyes and the mouth for drowsiness detection
    cv2.imshow("frame",frame)
    if cv2.waitKey(33) & 0xFF==ord('q'):
        break
webcam_on.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 79ms/step
